In [1]:
!pip install datasets
!pip install scikit-learn
!pip install ibm-watson-machine-learning==1.0.312

In [2]:
import os, getpass
from pandas import read_csv

In [3]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "ARULBypitCg6liH-9n7MLnP-d7b50anbnA4YtUsonw5-"
}

In [4]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your Project_ID (hit enter): ")

In [5]:
project_id

'b42556a3-9788-4c09-a272-2807738016cb'

In [6]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='K_iylTzBQahcovWR0R1HcBL2WbtfaIR0xiAJ_Rjsu5tV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'collegefeedbackclassifier-donotdelete-pr-ejz40fmj7wizdg'
object_key = 'feedack_train_1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

train_data = pd.read_csv(body)
train_data.head(10)

,Student_ID,Feedback,Theme
0,1,Professors often cancel lectures last-minute w...,Academics
1,92,No filtered water stations in science buildings,Facilities
2,10,The grading rubric for essays is vague and inc...,Academics
3,198,Student workers classified as ?volunteers? to ...,Administration
4,192,No translation services for non-English speaki...,Administration
5,193,Student records accidentally exposed in public...,Administration
6,94,Library study pods constantly reserved by same...,Facilities
7,90,Disability access ramps are too steep in the h...,Facilities
8,200,Thesis submission deadlines different per depa...,Administration
9,31,3-hour lectures without breaks violate univers...,Academics


In [7]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='K_iylTzBQahcovWR0R1HcBL2WbtfaIR0xiAJ_Rjsu5tV',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'collegefeedbackclassifier-donotdelete-pr-ejz40fmj7wizdg'
object_key = 'feedack_test_1.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

test_data = pd.read_csv(body)
test_data.head(10)


,Student_ID,Feedback,Theme
0,1,Mandatory course evaluations block registratio...,Academics
1,21,Peer tutoring center lacks subject specialists...,Academics
2,15,TA-led discussion sections contradict professo...,Academics
3,86,Student death protocols release information to...,Administration
4,20,Academic probation excludes students from rese...,Academics
5,18,Required 'textbook' is professor's unpublished...,Academics
6,79,Immigration documents processed after OPT dead...,Administration
7,78,FERPA violations occur in departmental mailings,Administration
8,81,Mental health leave requires course withdrawal...,Administration
9,34,Microbiology lab lacks negative pressure rooms,Facilities


In [8]:
train_data.shape

(210, 3)

In [9]:
test_data.shape

(90, 3)

In [10]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

In [11]:
model_id = ModelTypes.FLAN_T5_XXL

In [12]:
instruction = """Find the theme fo the college student feedback.
Choose the theme from the following list:
'Academics' , 'Facilities' , 'Administration'. \n """

In [13]:
few_shot_example=[]
few_shot_examples=[]
for i in range(5):
    few_shot_example.append(f"Feedback:\t{train_data.values[i][1]}\n\tTheme:\t{train_data.values[i][2]}\n")

few_shot_example.append("Feedback:\t")
few_shot_examples = [ "".join(few_shot_example) ]


In [14]:
few_shot_examples

['Feedback:\tProfessors often cancel lectures last-minute without rescheduling\n\tTheme:\tAcademics\nFeedback:\tNo filtered water stations in science buildings\n\tTheme:\tFacilities\nFeedback:\tThe grading rubric for essays is vague and inconsistent across TAs\n\tTheme:\tAcademics\nFeedback:\tStudent workers classified as ?volunteers? to avoid pay\n\tTheme:\tAdministration\nFeedback:\tNo translation services for non-English speaking parents\n\tTheme:\tAdministration\nFeedback:\t']

In [15]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.MAX_NEW_TOKENS: 5,
    GenParams.MIN_NEW_TOKENS: 1
}

In [16]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id = model_id,
    params = parameters,
    credentials = credentials,
    project_id = project_id)

In [17]:
import time
results = []
feedbacks = []
theme  = []

for i in range(10):
    feedbacks.append(test_data.values[i][1])
    theme.append(test_data.values[i][2])

for inp in feedbacks:
    results.append(model.generate(" ".join([instruction + few_shot_examples[0], inp, "\n\tTheme:\t"]))["results"][0])
    time.sleep(0.6)


In [18]:
feedbacks

['Mandatory course evaluations block registration until completed',
 'Peer tutoring center lacks subject specialists for STEM',
 "TA-led discussion sections contradict professors' lecture content",
 'Student death protocols release information too slowly',
 'Academic probation excludes students from research opportunities',
 "Required 'textbook' is professor's unpublished manuscript",
 'Immigration documents processed after OPT deadlines',
 'FERPA violations occur in departmental mailings',
 'Mental health leave requires course withdrawal penalty',
 'Microbiology lab lacks negative pressure rooms']

In [19]:
theme

['Academics',
 'Academics',
 'Academics',
 'Administration',
 'Academics',
 'Academics',
 'Administration',
 'Administration',
 'Administration',
 'Facilities']

In [20]:
for output in results:
    print(output["generated_text"])

Administration
Academics
Academics
Administration
Academics
Academics
Administration
Administration
Administration
Facilities
